In [ ]:
from fenics import *
import numpy as np

# --- 1. Definição de Parâmetros Físicos ---
k_saudavel = 0.5
k_tumoral = 0.55
wb_saudavel = 0.00051
wb_tumoral = 0.00125
Qm_saudavel = 420.0
Qm_tumoral = 4200.0
rhob = 1000.0
cb = 4200.0
Ta = 37.0
r0 = 3.1e-3

# --- Expressões e Classes de Ajuda ---
# Classe para mapear IDs da malha para valores de propriedades
class PropriedadeExpression(UserExpression):
    def __init__(self, subdomains, values, **kwargs):
        super().__init__(**kwargs)
        self.subdomains = subdomains
        self.values = values
    
    def eval_cell(self, values, x, cell):
        subdomain_id = self.subdomains[cell.index]
        values[0] = self.values.get(subdomain_id, self.values[1])
        
    def value_shape(self):
        return ()

# Classe para definir a fonte de calor do laser
class QrExpression(UserExpression):
    def __init__(self, pontos_laser, A_val, r0, **kwargs):
        super().__init__(**kwargs)
        self.pontos_laser = pontos_laser
        self.A_val = A_val
        self.r0 = r0

    def eval(self, values, x):
        total_qr = 0
        for x0, y0 in self.pontos_laser:
            r2 = (x[0] - x0)**2 + (x[1] - y0)**2
            total_qr += self.A_val * np.exp(-r2 / self.r0**2)
        values[0] = total_qr
    
    def value_shape(self):
        return ()

# --- Carregamento da Malha e Definição das Condições ---
mesh = Mesh("malha.xml")
subdomains = MeshFunction("size_t", mesh, "malha_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "malha_facet_region.xml")

V = FunctionSpace(mesh, "Lagrange", 1)

# Mapeia as IDs das sub-regiões para as propriedades físicas
k = PropriedadeExpression(subdomains, {1: k_saudavel, 2: k_tumoral})
wb = PropriedadeExpression(subdomains, {1: wb_saudavel, 2: wb_tumoral})
Qm = PropriedadeExpression(subdomains, {1: Qm_saudavel, 2: Qm_tumoral})

T = TrialFunction(V)
v = TestFunction(V)
dx = Measure('dx', domain=mesh, subdomain_data=subdomains)

left_boundary_id = 4
bc = DirichletBC(V, Constant(Ta), boundaries, left_boundary_id)

# --- Função para executar as simulações ---
def run_simulation(T, v, dx, bc, pontos_laser, A_val, output_filename):
    a = k * dot(grad(T), grad(v)) * dx + rhob * cb * wb * T * v * dx
    L = (rhob * cb * wb * Ta + Qm + QrExpression(pontos_laser, A_val, r0)) * v * dx
    
    T = Function(V)
    solve(a == L, T, bc)

    print(f"\n--- Simulação Concluída: {output_filename} ---")
    print(f"Temperatura Mínima: {T.vector().min():.2f} °C")
    print(f"Temperatura Máxima: {T.vector().max():.2f} °C")
    
    vtkfile = File(output_filename)
    vtkfile << T

# Executar a simulação com 1 ponto de injeção
ponto_laser_1p = [(0.05, 0.05)]
A_1ponto = 1.3e6
run_simulation(T, v, dx, bc, ponto_laser_1p, A_1ponto, 'solucao_um_ponto.pvd')

# Executar simulação com 4 pontos de injeção
pontos_laser_4p = [(0.045, 0.045), (0.055, 0.045), (0.045, 0.055), (0.055, 0.055)]
A_4pontos = 0.325e6
run_simulation(T, v, dx, bc, pontos_laser_4p, A_4pontos, 'solucao_quatro_pontos.pvd')